In [1]:
# fmt: off
%matplotlib inline
%config InlineBackend.figure_format = "retina"
%load_ext autoreload
%load_ext jupyter_black
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pickle
from mls_scf_tools.mls_pint import ureg
plt.style.use("mls_scf")
def vdir(obj):
    return [x for x in dir(obj) if not x.startswith('__')]


In [2]:
import intervaltree
import itertools
import pint
from matplotlib import colormaps
import warnings
import json
import pathlib
from matplotlib.backends.backend_pdf import PdfPages
import re

In [3]:
%autoreload 2
import njl_corf
from njl_corf.pyfcctab import ureg
import njl_corf.pyfcctab as pyfcctab
import njl_corf.pyoscar as pyoscar
import njl_corf.band_figure as band_figure
import njl_corf.wrc27_views as wrc27_views

In [4]:
allocation_database = pyfcctab.read(skip_additionals=False)

Reading tables: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, done.
Digesting: R1, R2, R3, F, NF, done.
Injecting additions in: R1, R2, R3, F, NF, done.
Merging: ITU, USA, all, done.
Footnote definitions: reading, appending, done.


In [5]:
print(allocation_database.r1[35.51 * ureg.GHz][0].bounds)

[<Quantity(35.5, 'gigahertz')>, <Quantity(36.0, 'gigahertz')>]


In [6]:
oscar_database = pyoscar.read()
oscar_database = pyoscar.merge_sensors(
    oscar_database,
    {
        "AMSR": "AMSR*",
        "ATMS": "ATMS*",
        "AWS": "AWS*",
        "MWTS": "MWTS*",
        "SSMIS": "SSMIS*",
        "AMSU": "AMSU*",
        #        "MWRI": "MWRI*",
        "MWS": "MWS*",
        "MTVZA": "MTVZA*",
        "MWI": "MWI*",
        "SRAL": "SRAL*",
        "OSCAT": "OSCAT*",
        "Altimetry": ["ALT*", "RA*", "Altimeter*"],
        "Precipitation": ["PR*", "DPR*", "Rainradar"],
        "Poseidon": "Poseidon*",
        "SWIM": "SWIM*",
        "MWR": "MWR*",
        "WSF": "WSF*",
        "MLS": "MLS*",
        "ICI": "ICI*",
    },
)

In [7]:
config = band_figure.FigureConfiguration.load()

In [8]:
collation_rules = {
    "Space research": ["Space research*"],
    "Mobile": ["Mobile", "Mobile*"],
    "Fixed": ["Fixed", "Fixed*"],
    "Aeronautical": ["Aeronautical*"],
    "EESS (passive)": ["Earth exploration-satellite *(passive)"],
    "EESS (active)": ["Earth exploration-satellite *(active)"],
    "EESS (comms)": ["Earth exploration-satellite *(*to*)"],
    "Amateur": ["Amateur*"],
    "Radionavigation": ["Radionavigation*"],
    "Meteorological": ["Meteorological*"],
    "Space operation": ["Space operation*"],
    "Radiodetermination": ["Radiodetermination*"],
    "Radio astronomy": ["Radio Astronomy*"],
}

In [9]:
ai_ranges = wrc27_views.get_ai_info()

In [16]:
def all_wrc27_figures():
    config = band_figure.FigureConfiguration.load()
    ureg.setup_matplotlib()
    with PdfPages("example-ai-figures.pdf") as pdf:
        for range_name, range_details in ai_ranges.items():
            print(f"{range_name}: {range_details}")
            if range_details.frequency_band.start is None:
                continue
            fig, ax = plt.subplots(figsize=[8.5, 11], layout="constrained")
            band_figure.views_plot(
                frequency_range=range_details,
                allocation_database=allocation_database,
                oscar_database=oscar_database,
                ax=ax,
                figure_configuration=config,
                collation_rules={},  # collation_rules,
                omit_band_borders=True,
            )
            ax.set_title(range_name)
            pdf.savefig(fig)
            plt.close()


all_wrc27_figures()

WRC-27 AI-1.1a: AgendaItemSubBand(name='AI-1.1', wrc='WRC-27', text=None, soundbyte='ESIMs (aero/marine)', frequency_band=slice(<Quantity(47.2, 'gigahertz')>, <Quantity(50.2, 'gigahertz')>, None), i_band=0, n_bands=2)
WRC-27 AI-1.1b: AgendaItemSubBand(name='AI-1.1', wrc='WRC-27', text=None, soundbyte='ESIMs (aero/marine)', frequency_band=slice(<Quantity(50.4, 'gigahertz')>, <Quantity(51.4, 'gigahertz')>, None), i_band=1, n_bands=2)
WRC-27 AI-1.2a: AgendaItemSubBand(name='AI-1.2', wrc='WRC-27', text=None, soundbyte='FSS uplinks', frequency_band=slice(<Quantity(13.75, 'gigahertz')>, <Quantity(14.0, 'gigahertz')>, None), i_band=0, n_bands=1)
WRC-27 AI-1.3a: AgendaItemSubBand(name='AI-1.3', wrc='WRC-27', text=None, soundbyte='FSS NGSO uplinks', frequency_band=slice(<Quantity(51.4, 'gigahertz')>, <Quantity(52.4, 'gigahertz')>, None), i_band=0, n_bands=1)
WRC-27 AI-1.4a: AgendaItemSubBand(name='AI-1.4', wrc='WRC-27', text=None, soundbyte='FSS NGSO downlinks', frequency_band=slice(<Quantity(1

In [ ]:
test = band_figure.views_plot(
    frequency_range=slice(1300 * ureg.MHz, 1450 * ureg.MHz),
    allocation_database=allocation_database,
    oscar_database=oscar_database,
    figure_configuration=config,
    collation_rules=collation_rules,
    frequency_margin=200 * ureg.MHz,
)

In [ ]:
allocation_database.itu.find_closest_matching_band(
    frequency=1475 * ureg.MHz,
    direction=1,
    condition=lambda band: band.has_allocation("Radio Astronomy"),
)

In [ ]:
def make_html():
    for ai in ai_ranges.keys():
        wrc27_views.ai_html_summary(
            ai,
            allocation_database=allocation_database,
            oscar_database=oscar_database,
        )


make_html()

In [ ]:
wrc27_views.generate_index_html()

In [ ]:
import pathlib
import shutil

In [ ]:
# wrc27_views.push_information()